In [76]:
import pandas as pd
import pickle

In [38]:
# Imports
import json
import logging
import statistics
import sys
from typing import List

import openai
from openai.types.chat import (
    chat_completion,
    chat_completion_message,
)
import sklearn.metrics

In [68]:
# One quick thing we can do with the client is confirm we have credits.
credit_balance = client.organization.get_credit_balance()
print(credit_balance)

OrganizationBalance(credits=50.0)


In [41]:
df = pd.read_parquet("MMLU-PIECE.parquet")
df

,answer,choices,question,subject
0,1,"[Adams only., Brooks only., Case only., Adams ...",Davis decided to kill Adams. He set out for Ad...,
1,3,"[guilty, because this is a public welfare offe...",A state statute requires any person licensed t...,
2,2,"[Yes, because Mann threatened to use deadly fo...","Lender met Borrower on the street, demanded th...",
3,1,[must permit Don to answer if he had objected ...,Peter sued Don for breach of contract. The cou...,
4,2,[partial breach of contract only if Ames had p...,Ames had painted Bell's house under a contract...,
...,...,...,...,...
99837,0,"[skyscrapers, the stock market, air, water]",A bulldozer alters the area of,
99838,0,"[Brewer's yeast, air, sand, sugar]",An organism that can survive without the help ...,
99839,1,"[the praying mantis, the antelope, the butterf...",The nimbleness of this animal is a key adaptio...,
99840,0,"[organisms they hunt, computer, groceries, seven]",Birds will have different kinds of beaks depen...,


In [42]:
df["chosen"] = df.apply(lambda r: r["choices"][r["answer"]], axis=1)

In [43]:
df

,answer,choices,question,subject,chosen
0,1,"[Adams only., Brooks only., Case only., Adams ...",Davis decided to kill Adams. He set out for Ad...,,Brooks only.
1,3,"[guilty, because this is a public welfare offe...",A state statute requires any person licensed t...,,"not guilty, because he was not aware of the va..."
2,2,"[Yes, because Mann threatened to use deadly fo...","Lender met Borrower on the street, demanded th...",,"No, if it was apparent that Lender was about t..."
3,1,[must permit Don to answer if he had objected ...,Peter sued Don for breach of contract. The cou...,,"may permit Don to answer, whether or not he ha..."
4,2,[partial breach of contract only if Ames had p...,Ames had painted Bell's house under a contract...,,total breach of contract only if Ames had prop...
...,...,...,...,...,...
99837,0,"[skyscrapers, the stock market, air, water]",A bulldozer alters the area of,,skyscrapers
99838,0,"[Brewer's yeast, air, sand, sugar]",An organism that can survive without the help ...,,Brewer's yeast
99839,1,"[the praying mantis, the antelope, the butterf...",The nimbleness of this animal is a key adaptio...,,the antelope
99840,0,"[organisms they hunt, computer, groceries, seven]",Birds will have different kinds of beaks depen...,,organisms they hunt


In [74]:
# Run the judge spec.
arr=[]
for i in range(1,100):
    chat_request_text = str(df["question"][i])
    chat_response_text = str(df["chosen"][i])

    completion_request = {
        "model": llm_models.GPT_4O_MINI,
        "messages": [{"role": "user", "content": chat_request_text}],
    }
    chat_completion_response = chat_completion.ChatCompletion(
        id="123",
        choices=[
            chat_completion.Choice(
                finish_reason="stop",
                index=0,
                message=chat_completion_message.ChatCompletionMessage(
                    role="assistant",
                    content=chat_response_text,
                ),
            )
        ],
        created=0,
        model="gpt-4o",
        object="chat.completion",
        service_tier=None,
    )

    evaluation_result = client.judges.evaluate_using_judge_spec(
        rubric_judge_spec.to_dict(),
        completion_request=completion_request,
        completion_response=chat_completion_response,
    )
    arr.append(evaluation_result)

# print(f"User: {chat_request_text}")
# print(f"Assistant: {chat_response_text}")
# print(f"Evaluation result: {evaluation_result}")

In [ ]:
arr


[JudgeEvaluation(score=4, reason='The assistant\'s response is reasonably accurate. The statute specifies that it is a misdemeanor to "knowingly fail to file" the report. As per the scenario, Nelson claims he did not realize his sales exceeded the threshold, indicating a lack of knowledge about the requirement to file the report. If the jury believes Nelson\'s testimony, they might reasonably conclude that he did not "knowingly" fail to file the report, which aligns with the assistant\'s conclusion. However, the answer could be expanded for clarity by explaining the concept of "knowingly" in legal terms regarding mens rea. Therefore, while the response is correct, it could benefit from a more detailed legal explanation.', cost=0.002455),
 JudgeEvaluation(score=3, reason='\nThe assistant\'s response provides a concise legal analysis of the situation described. It effectively addresses whether Lender will prevail in an assault claim against Mann. The key aspect of the situation is whethe